# Coleta das bases usadas na análise

Nome: Ellen Beatriz Shen

Nome: Gabriela Moreno Boriero

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

## Retirando os tweets

In [ ]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

In [ ]:
#Produto escolhido:
produto = 'Johnny Depp'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

## Junção de duas bases

- já apagamos as duas bases que usamos para a base final, mas aqui está o código que usamos.

In [ ]:
#ler os dois excel
excel_1 = pd.read_excel("acai_1.xlsx")
excel_2 = pd.read_excel("acai_2.xlsx")

In [ ]:
#retirar as repeticoes da coluna de treinamento
sem_repeticao_1 = set(excel_1.Treinamento)
sem_repeticao_2 = set(excel_2.Treinamento)

In [ ]:
#transformar em uma série
series_1 = pd.Series(list(sem_repeticao_1)) #após o set, precisa enviar como uma lista, por isso o list()
series_2 = pd.Series(list(sem_repeticao_2))

In [ ]:
#juntar as series com o pd.concat()
series_juntado = pd.concat([series_1,series_2])

In [ ]:
#retirar possíveis duplicaçoes novamente
base_final = set(series_juntado)

#voltar para uma serie
series_final = pd.Series(list(base_final))

In [ ]:
#criar o excel final
series_final.to_excel("base_acai.xlsx")

## Transformar o excel em um arquivo .txt

In [15]:
base_acai = pd.read_excel("base_acai.xlsx")

In [16]:
acai_gosta = base_acai[base_acai.Classe == 0]
acai_indiferente = base_acai[base_acai.Classe == 1]
acai_nao_gosta = base_acai[base_acai.Classe == 2]

In [17]:
series_gosta = pd.Series(list(acai_gosta.Treinamento))
series_indiferente = pd.Series(list(acai_indiferente.Treinamento))
series_nao_gosta = pd.Series(list(acai_nao_gosta.Treinamento))

In [20]:
gosta = []
for tweets in series_gosta:
    gosta.append(tweets)

gosta_texto=', '.join(gosta)

with open('gosta.txt','w', encoding="utf8") as arquivo:
    arquivo.write(gosta_texto)

In [18]:
indiferente = []
for tweets in series_indiferente:
    indiferente.append(tweets)
    
indiferente_texto=', '.join(indiferente)

with open('indiferente.txt','w', encoding="utf8") as arquivo:
    arquivo.write(indiferente_texto)

In [21]:
nao_gosta = []
for tweets in series_nao_gosta:
    nao_gosta.append(tweets)
    
nao_gosta_texto=', '.join(nao_gosta)

with open('nao_gosta.txt','w', encoding="utf8") as arquivo:
    arquivo.write(nao_gosta_texto)